In [1]:
import urllib
import json
import pandas as pd
import numpy as np
import warnings
import pickle
from datetime import datetime
from datetime import timedelta
import statistics
warnings.filterwarnings('ignore')

###### LOAD

In [2]:
FILE_NAME = "../original_datasets/tableau_time_sereis0403.csv" 
data = pd.read_csv(FILE_NAME,  comment='"', header=0 )  #encoding="ISO-8859-1",, index_col=[9,10]
data=data.sort_values('Date') #.sort_index(level=0)
print ('len1',len(data))   
data =data.loc[data['Country_Region']!='Cruise Ship'] 
print ('len2',len(data))  
mask =( data['Cases']!= 0  )
data  =data.loc[mask ]
print ('len3',len(data))
data=data.sort_values([ 'Lat','Long'])
mask=(data['Lat']==0   )
bad=data.loc[mask] 
print ('Number of rows with no Lat or Long', len(bad) )
mask=(data['Lat']!=0  )
datax=data.loc[mask]
print ('len4',len(datax))
valid_dates=[]
printrun=0
valid_data=[]
for row in datax.iterrows():
    this_date=row[1]['Date'] 
    if this_date in valid_dates:
        pass
    else:
        valid_dates.append(this_date)
    this_data = [ row[1]['Lat'], row[1]['Long'], row[1]['Country_Region'], row[1]["Province_State"] ] 
                                                                            # cases and difference later    
    if this_data[0]!=np.nan :
        if this_data in valid_data:
            pass
        else:
            if  (   this_data[0] <0 ) | (  this_data[0] >0 ):
                valid_data.append(this_data)
            else: pass # print ('disregarded' , this_data)
print ('valid dates len' , len(valid_dates) ) #,valid_dates)
print ('valid data len' , len(valid_data) ) #,valid_data, 
good_dates=[]
for date in valid_dates:
    splited=date.split( '/')
    if len(splited[0])<2 : splited[0]='0'+splited[0]
    if len(splited[1])<2 : splited[0]='0'+splited[1]
    ref_date=splited[2]+'/'+splited[0]+'/'+splited[1]  # for sort
    good_dates.append(ref_date)
sorted_dates=sorted(good_dates)
for i in range(len(sorted_dates)):
    splited=sorted_dates[i].split( '/')
    sorted_dates[i]=splited[1]+'/'+splited[2]+'/'+splited[0]  # return the sorted list to the file format
BIG= pd.DataFrame (valid_data ,columns= ['Lat' ,'Long','Country_Region',"Province_State"])
BIG=BIG.set_index(['Lat','Long'])

len1 113510
len2 113366
len3 44164
Number of rows with no Lat or Long 345
len4 43819
valid dates len 72
valid data len 2584


,,Country_Region,Province_State
Lat,Long,,
-41.454500,145.970700,Australia,Tasmania
-40.900600,174.886000,New Zealand,NaN
-38.416100,-63.616700,Argentina,NaN
-37.813600,144.963100,Australia,Victoria
-35.675100,-71.543000,Chile,NaN
-35.473500,149.012400,Australia,Australian Capital Territory
-34.928500,138.600700,Australia,South Australia
-33.868800,151.209300,Australia,New South Wales
-32.522800,-55.765800,Uruguay,NaN


In [3]:
cols=2
rows=len(BIG)
for date in sorted_dates:
    BIG.insert(cols,date,np.zeros(rows),allow_duplicates=True)
    cols+=1

In [4]:
BIG2=BIG.copy()
count=0
for row in datax.iterrows():
    count+=1
    line=row[1]
    if line['Case_Type']!='Confirmed': continue
    if line['Cases']==0 | (not ( (line['Cases']>0) |(  line ['Cases']<0))) : continue
    if line['Lat']==0: continue
    if line['Long']==0: continue
  
    #print ('types',type(line['Lat'] ))
    #print ('keys',line['Lat'],line[ 'Long'] )
      
    try:
        date2=line['Date'].split('/')  # Some date are missing 0 on left
        if len(date2[0])<2: date2[0]='0'+date2[0]
        if len(date2[1])<2: date2[1]='0'+date2[1]
        date3=date2[0]+'/'+date2[1]+'/'+date2[2]
        BIG2.loc[( line['Lat'], line['Long']), date3]=line['Cases']
    except:
        print ('KEY OR INDEX ERROR for lat loc date cases', line['Lat'], line['Long'] , date3,line['Cases'])
    if count  % 500 ==0: 
        print ('BIG2 updated,count, lat, long,date, cases', count,line['Lat'], line['Long'] , date3,line['Cases'])

BIG2 updated,count, lat, long,date, cases 1000 -12.4634 130.8456 03/10/2020 1.0
BIG2 updated,count, lat, long,date, cases 1500 1.2833 103.8333 04/02/2020 1049.0
BIG2 updated,count, lat, long,date, cases 2000 7.54 -5.5471 03/25/2020 80.0
BIG2 updated,count, lat, long,date, cases 2500 12.5186 -70.0358 03/23/2020 9.0
BIG2 updated,count, lat, long,date, cases 3500 18.2208 -66.5901 03/29/2020 127.0
BIG2 updated,count, lat, long,date, cases 4000 21.9162 95.956 03/27/2020 8.0
BIG2 updated,count, lat, long,date, cases 5000 25.3548 51.1839 03/14/2020 337.0
BIG2 updated,count, lat, long,date, cases 5500 26.8154 106.8748 02/19/2020 146.0
BIG2 updated,count, lat, long,date, cases 6000 27.614 115.7221 03/07/2020 935.0
BIG2 updated,count, lat, long,date, cases 6500 29.1832 120.0934 01/31/2020 538.0
BIG2 updated,count, lat, long,date, cases 7500 30.20406249 -90.91328409999998 03/30/2020 153.0
BIG2 updated,count, lat, long,date, cases 8500 30.87454847 -84.23418364 04/02/2020 2.0
BIG2 updated,count, la

KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/25/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/26/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/26/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/26/2020 nan
KEY OR INDEX ERROR for la

KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
BIG2 updated,count, lat, long,date, cases 38000 nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR 

KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for la

KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 03/31/2020 nan
KEY OR INDEX ERROR for la

...

KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for la

KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for la

KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 37.0
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for l

KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for la

KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for lat loc date cases nan nan 04/02/2020 nan
KEY OR INDEX ERROR for la

In [5]:
BIG3=BIG2.copy()
clen=len(BIG3.columns)
rlen=len(BIG3)
BIG3.insert(2,'First_7',np.zeros(rlen),allow_duplicates=True)
BIG3.insert(2,'5%_Date',rlen*pd.Series([np.nan]),allow_duplicates=True)
BIG3.insert(2,'Last relevant date',rlen*pd.Series([np.nan]),allow_duplicates=True)
BIG3.insert(2,'GFQ3',np.zeros(rlen),allow_duplicates=True)
BIG3.insert(2,'GFQ2',np.zeros(rlen),allow_duplicates=True)
BIG3.insert(2,'GFQ1',np.zeros(rlen),allow_duplicates=True)
BIG3

,,Country_Region,Province_State,GFQ1,GFQ2,GFQ3,Last relevant date,5%_Date,First_7,01/1/2020,01/1/2020,...,03/01/2020,02/01/2020,02/02/2020,02/03/2020,02/04/2020,02/05/2020,02/06/2020,02/07/2020,02/08/2020,02/09/2020
Lat,Long,,,,,,,,,,,,,,,,,,,,,
-41.454500,145.970700,Australia,Tasmania,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-40.900600,174.886000,New Zealand,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-38.416100,-63.616700,Argentina,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-37.813600,144.963100,Australia,Victoria,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,7.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
-35.675100,-71.543000,Chile,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-35.473500,149.012400,Australia,Australian Capital Territory,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-34.928500,138.600700,Australia,South Australia,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
-33.868800,151.209300,Australia,New South Wales,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,6.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
-32.522800,-55.765800,Uruguay,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
BIG4=BIG3.copy()
datarows = [row for row in BIG4.iterrows()  ]
#hopkins_confirmed.insert(7,column='last relevant date',value=np.nan)

 


### Obtain a list of dates (string of columns names which are dates , assumed to be a continouos sequence in the DF)


cur=0


valid_dates=sorted_dates  # hopkins_confirmed.columns[cur:rvrscur]         # valid dates - all the dates in the index

 

for row in datarows:
    #print (row[0],row[0][1])
    ratios=[0]
    totals=[]
    actual_dates=[]
    
    for cursor in range (len(valid_dates)-1):
        p=row[1][valid_dates[cursor]]
        q=row[1][valid_dates[cursor+1]  ]
        actual_dates.append(valid_dates[cursor])
        if p.values[0] > 0:
            ratios.append (round(q/p - 1,4 ))
            totals.append (p)
        else:
            if q>0:
                ratios.append (4)           # the first gr is set to 4 (5 times ...) and will most often get removed due to
                                            # totals less then 5 pct
            else: ratios.append(0)          #  0 growth between two zeros...
            totals.append(0)
            
    #ratios=[0]+ratios  # so that each growth rate 'sits' under its 'total' and is the growth between this and the 
                       # previous
    totals.append(row[1][valid_dates[-1]]) # add the last total to list
    actual_dates.append(valid_dates[-1])
           
    print ('==========')        
    print  ('new',row[1]['Province_State'],row[1][ 'Country_Region'])
    print ('totals',totals)
    print ('ratios',ratios)
       
    if max(totals)<20:
        print ('SKIP')
        continue    ###  if no day with 20 or more, skip this region,  small numbers 
    print ('NO SKIP')
        
   
    print (row[1]['Province_State'],row[1][ 'Country_Region'],ratios,totals)
    print ('lens',len(ratios),len(totals))
    maxt=max(totals)
    few=maxt/20
    print (row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    # here , find the 5% seperator
    few_seperator = -1
    for cursor in range (len(totals)-1):
        if  (totals[cursor] <=few ) and (totals [cursor+1]>=few):
            few_seperator=cursor
            break
    if few_seperator >=0 :
        t_sample=totals[cursor+1:]
        t_ratios=ratios[cursor+1:]
        actual_dates=actual_dates[cursor+1:]
        five_pct_name=actual_dates[0]
        
        print ('five pct', five_pct_name) 
    else:
        five_pct_name=actual_dates[0]  # this is the complete actual_dates
        print ('five pct not found',five_pct_name )
 
    #   find if there is a sequence of growth rate lower than 2% per day (from the end )  remove it if found
    
    for cur in range (len(t_ratios)):
        maxright=max(t_ratios[cur:])
        if   (maxright>0.02):
            continue
        else:
            print ('maxright ,cur, len ratios, rightside',maxright,cur,len(t_ratios),t_ratios[cur:],t_sample[cur:] )
            break
    
    
    last_relevant_sample=actual_dates[ -1]  # overriden if cut   
    if cur < len(t_ratios)-1:
     
        print ('cut right side len befor , len after',len(t_ratios),cur+1)
        
        t_ratios=t_ratios [:cur+1]
        t_sample=t_sample [:cur+1]
        actual_dates=actual_dates[:cur+1]
        last_relevant_sample=actual_dates[ -1]
        
    else: print ('nothing was cut')
            
    
    sorted_sample=t_sample.sort()
    
    
    print ('sample size',len(t_sample),t_sample,t_ratios)
    
    med=statistics.median(t_ratios)
    
    lower=np.percentile(t_ratios,25)
    upper=np.percentile(t_ratios,75)
    
    print ('lower, median, upper',lower,med,upper)
    
     
    daycount=0
    sevendays=[]
    
    for smpl in t_sample:
        if smpl< few : continue
        else:
            daycount +=1
            if daycount==8 : break
            else: sevendays.append(smpl)
    if daycount==8:
        if sevendays[0]>0:
            
            seven_days_growth=(sevendays[-1]/sevendays[0])-1
    
        else:
            daycount=0
    else: daycount=0
     
    
    coord=row[0][0:2]  #coord=row [0][0]
    if daycount>0:
        BIG4.loc[coord, 'First_7']  = seven_days_growth
         
        print ( 'Debug 7 ', seven_days_growth)
        
    BIG4.loc[coord, '5%_Date']  = five_pct_name
    BIG4.loc[coord, 'last relevant date'] =last_relevant_sample
    BIG4.loc[coord, 'GFQ1'] =lower
    BIG4.loc[coord, 'GFQ2'] =med
    BIG4.loc[coord, 'GFQ3'] =upper
    print ('Debug 0 ',BIG4.loc[coord,'GFQ1'] , BIG4.loc[coord,'GFQ3'] )
    
     
#BIG3.insert(2,'First_7',np.zeros(rlen),allow_duplicates=True)
#BIG3.insert(2,'5%_Date',rlen*pd.Series([np.nan]),allow_duplicates=True)
##BIG3.insert(2,'Last relevant date',rlen*pd.Series([np.nan]),allow_duplicates=True)
#BIG3.insert(2,'GFQ3',np.zeros(rlen),allow_duplicates=True)
#BIG3.insert(2,'GFQ2',np.zeros(rlen),allow_duplicates=True)
#BIG3.insert(2,'GFQ1',np.zeros(rlen),allow_duplicates=True)
#BIG3
    print ('DEBUG1' , row[1]['Province_State'],row[1][ 'Country_Region'],maxt,few)
    print ('DEBUG2' , lower, med, upper,five_pct_name,len(actual_dates),len(t_ratios))
    print ('DEBUG3', t_ratios)
    print ('DEBUG4', t_sample)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
p.values[0]

0.0

###### Calculate growth factors

In [119]:
#hopkins_confirmed 
#hopkins_confirmed.columns



#dataonly=hopkins_confirmed['information']=='data'
#hopconf_data=hopkins_confirmed[dataonly]
#hopconf_data

Province_State  \
coordinate                     information                                  
(-41.4545, 145.9707)           data                              Tasmania   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-40.9006, 174.886)            data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-38.4161, -63.6167)           data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-37.8136, 144.9631)           data                              Victoria   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-35.6751, -71.543)            data                                   NaN   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
(-35.4735, 149.0124)           data          Australian Capital Territory   
                               avg_d_RH                               NaN   
                               avg_d_precip                           NaN   
                               avg_d_tmp                              NaN   
                               avg_d_wind                             NaN   
...                                                                   ...   
(48.3794, 31.1656)             Growth rates                           NaN   
(48.669, 19.699)               Growth rates                           NaN   
(49.2827, -123.1207)           Growth rates                           NaN   
(49.3723, -2.3644)             Growth rates                           NaN   
(49.8153, 6.1296)              Growth rates                           NaN   
(49.8175, 15.472999999999999)  Growth rates                           NaN   
(50.8333, 4.0)                 Growth rates                           NaN   
(51.0, 9.0)                    Growth rates                           NaN   
(51.2538, -85.3232)            Growth rates                           NaN   
(51.9194, 19.1451)             Growth rates                           NaN   
(52.1326, 5.2913)              Growth rates                           NaN   
(52.9399, -106.4509)           Growth rates                           NaN   
(52.9399, -73.5491)            Growth rates                           NaN   
(53.1355, -57.6604)            Growth rates                           NaN   
(53.1424, -7.6921)             Growth rates                           NaN   
(53.7098, 27.9534)             Growth rates                           NaN   
(53.7609, -98.8139)            Growth rates                           NaN   
(53.9333, -116.5765)           Growth rates                           NaN   
(54.2361, -4.5481)             Growth rates                           NaN   
(55.1694, 23.8813)             Growth rates            

###### Save file as pickle

In [227]:
#pickle_path = RESULT_PATH.format(datetime.now().strftime('%d%m'))
#with open(pickle_path, 'wb') as file:
#    pickle.dump(hopkins_confirmed, file)